In [ ]:
##Used single clip and augmented before sampling

In [1]:
import os
project_path = '/user/HS402/zs00774/Downloads/action-recognition-vit'  # Update with your project path
os.chdir(project_path)

In [2]:
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable


In [3]:
dataset_path = '/user/HS402/zs00774/Downloads/HMDB_simp'  # Update with your dataset path
if not os.path.exists(dataset_path):
    raise FileNotFoundError(f"Dataset not found at {dataset_path}")
else:
    print("Dataset found!")

Dataset found!


In [4]:
import torch
import numpy as np
import json
from transformers import Trainer, TrainingArguments,EarlyStoppingCallback
from transformers.trainer_utils import IntervalStrategy
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress TensorFlow warnings
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)  # Suppress FutureWarnings
from torch import nn, optim
from torch.utils.tensorboard import SummaryWriter
from torch.cuda.amp import GradScaler, autocast  # For mixed precision training
import logging
import sys
import argparse
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import train_test_split

from transformers import AutoImageProcessor, AutoModelForVideoClassification
import os
from transformers import TimesformerForVideoClassification

2025-08-06 21:48:04.838446: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-06 21:48:04.844772: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754513284.852910   18622 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754513284.855336   18622 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754513284.861612   18622 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [39]:
import os
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split

class MultiClipHMDBDataset(Dataset):
    def __init__(self, root_dir, clip_size=8, frame_rate=32, transform=None):
        """
        Virtual dataset that creates one clip per video with frame augmentation
        
        Args:
            root_dir (str): Path to dataset directory
            clip_size (int): Number of frames per clip
            frame_rate (int): Sampling rate (every Nth frame)
            transform: Torchvision transforms
        """
        self.root_dir = root_dir
        self.clip_size = clip_size
        self.frame_rate = frame_rate
        self.transform = transform
        self.data = self._load_data()

    def _augment_frames_sequence(self, frames_list, target_count):
        """
        Apply augmentation techniques in sequence until we reach target_count frames
        Check after each augmentation step and continue to next step only if needed
        Repeat the entire augmentation cycle if still insufficient
        1. Frame averaging (interpolation between consecutive frames)
        2. Temporal reversal
        3. Brightness variation
        """
        if len(frames_list) == 0:
            return ['black_frame'] * target_count
        
        augmented_frames = frames_list.copy()
        max_iterations = 5
        iteration = 0
        
        while len(augmented_frames) < target_count and iteration < max_iterations:
            iteration += 1
            
            current_frames_start = len(augmented_frames)
            
            # Step 1: Frame averaging (interpolation)
            if len(augmented_frames) < target_count and len(augmented_frames) >= 2:
                original_frames = augmented_frames.copy()
                for i in range(len(original_frames) - 1):
                    if len(augmented_frames) >= target_count:
                        break
                    frame1 = original_frames[i]
                    frame2 = original_frames[i + 1]
                    interp_frame = f"interp_{frame1}_{frame2}"
                    insert_pos = augmented_frames.index(frame1) + 1
                    augmented_frames.insert(insert_pos, interp_frame)
            
            # Check if target reached after frame averaging
            if len(augmented_frames) >= target_count:
                break
            
            # Step 2: Temporal reversal
            if len(augmented_frames) < target_count:
                original_sequence = augmented_frames.copy()
                reversed_sequence = original_sequence[::-1][1:]
                if reversed_sequence:
                    augmented_frames.extend(reversed_sequence)
            
            # Check if target reached after temporal reversal
            if len(augmented_frames) >= target_count:
                break
            
            # Step 3: Brightness variation
            if len(augmented_frames) < target_count:
                original_count = len(augmented_frames)
                brightness_levels = [0.6, 0.8, 1.2, 1.4]
                
                for brightness in brightness_levels:
                    if len(augmented_frames) >= target_count:
                        break
                    for i in range(original_count):
                        if len(augmented_frames) >= target_count:
                            break
                        original_frame = augmented_frames[i]
                        bright_frame = f"bright_{brightness}_{original_frame}"
                        augmented_frames.append(bright_frame)
            
            # Check if target reached after brightness variation
            if len(augmented_frames) >= target_count:
                break
            
            # Check if we made any progress in this iteration
            if len(augmented_frames) == current_frames_start:
                break
        
        return augmented_frames[:target_count]

    def _load_data(self):
        """Create single clip per video with new augmentation strategy"""
        data = []
        subfolders = sorted(os.listdir(self.root_dir))
        
        if len(subfolders) != 25:
            raise ValueError(f"Dataset must contain exactly 25 subfolders. Found {len(subfolders)}.")

        for label, action in enumerate(subfolders):
            action_path = os.path.join(self.root_dir, action)
            if not os.path.isdir(action_path):
                continue
                
            for video_folder in os.listdir(action_path):
                video_path = os.path.join(action_path, video_folder)
                if not os.path.isdir(video_path):
                    continue
                
                # Skip already processed folders
                if '_' in video_folder and video_folder.split('_')[-1].isdigit():
                    continue
                
                # Get all image frames (unsampled)
                all_frames = sorted([f for f in os.listdir(video_path) 
                                   if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))])
                
                if len(all_frames) == 0:
                    continue
                
                # Check if we need augmentation
                required_frames = self.frame_rate * self.clip_size  # e.g., 32 * 8 = 256
                
                if len(all_frames) < required_frames:
                    # Apply augmentation to get enough frames
                    augmented_frames = self._augment_frames_sequence(all_frames, required_frames)
                    working_frames = augmented_frames
                else:
                    working_frames = all_frames
                
                # Now sample from the working frames
                sampled_frames = working_frames[::self.frame_rate]
                
                # Create single clip per video
                if len(sampled_frames) >= self.clip_size:
                    # Take the first clip_size frames for the single clip
                    clip_frames = sampled_frames[:self.clip_size]
                    data.append((video_path, label, 0, clip_frames))
                else:
                    # This should be rare since we augmented first, but handle it
                    # Repeat frames to reach clip_size
                    while len(sampled_frames) < self.clip_size:
                        sampled_frames.extend(sampled_frames)
                    clip_frames = sampled_frames[:self.clip_size]
                    data.append((video_path, label, 0, clip_frames))
                
        print(f"Total clips created: {len(data)}")
        return data

    def __len__(self):
        return len(self.data)

    def _resolve_frame_path(self, video_path, frame_name):
        """Helper method to resolve frame path, handling brightness variants"""
        if frame_name.startswith('bright_'):
            parts = frame_name.split('_', 2)
            if len(parts) >= 3:
                original_frame = parts[2]
                frame_path = os.path.join(video_path, original_frame)
                if os.path.exists(frame_path):
                    return frame_path
        else:
            frame_path = os.path.join(video_path, frame_name)
            if os.path.exists(frame_path):
                return frame_path
        return None

    def _get_fallback_frame(self, video_path):
        """Get a fallback frame when specific frame loading fails"""
        available_frames = [f for f in os.listdir(video_path) 
                          if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
        if available_frames:
            return Image.open(os.path.join(video_path, available_frames[0]))
        else:
            return Image.fromarray(np.zeros((224, 224, 3), dtype=np.uint8))

    def _load_frames_from_clip(self, video_path, clip_frames):
        """Load frames for a specific clip"""
        frames = []
        
        for frame_name in clip_frames:
            if frame_name == 'black_frame':
                # Create black frame
                img = Image.fromarray(np.zeros((224, 224, 3), dtype=np.uint8))
            
            elif frame_name.startswith('interp_'):
                # Handle interpolated frames
                # Format: "interp_frame1_frame2"
                parts = frame_name.replace('interp_', '').split('_')
                if len(parts) >= 2:
                    frame1_name = '_'.join(parts[:-1])
                    frame2_name = parts[-1]
                    
                    frame1_path = os.path.join(video_path, frame1_name)
                    frame2_path = os.path.join(video_path, frame2_name)
                    
                    if os.path.exists(frame1_path) and os.path.exists(frame2_path):
                        try:
                            img1 = np.array(Image.open(frame1_path))
                            img2 = np.array(Image.open(frame2_path))
                            # Simple interpolation
                            img_avg = np.mean([img1, img2], axis=0, dtype=np.uint8)
                            img = Image.fromarray(img_avg)
                        except Exception as e:
                            print(f"Error interpolating frames: {e}")
                            img = Image.open(frame1_path)  # Fallback to first frame
                    else:
                        img = self._get_fallback_frame(video_path)
                else:
                    img = self._get_fallback_frame(video_path)
            
            elif frame_name.startswith('bright_'):
                # Handle brightness-varied frames - extract original frame and apply brightness
                # Format: "bright_1.2_original_frame.jpg"
                parts = frame_name.split('_', 2)  # Split into ['bright', '1.2', 'original_frame.jpg']
                if len(parts) >= 3:
                    brightness_factor = float(parts[1])
                    original_frame = parts[2]
                    
                    # Handle nested cases (bright of interp, bright of bright, etc.)
                    if original_frame.startswith('interp_'):
                        # This is a brightness variant of an interpolated frame
                        interp_parts = original_frame.replace('interp_', '').split('_')
                        if len(interp_parts) >= 2:
                            frame1_name = '_'.join(interp_parts[:-1])
                            frame2_name = interp_parts[-1]
                            
                            frame1_path = os.path.join(video_path, frame1_name)
                            frame2_path = os.path.join(video_path, frame2_name)
                            
                            if os.path.exists(frame1_path) and os.path.exists(frame2_path):
                                try:
                                    img1 = np.array(Image.open(frame1_path))
                                    img2 = np.array(Image.open(frame2_path))
                                    # Interpolate first
                                    img_avg = np.mean([img1, img2], axis=0, dtype=np.uint8)
                                    # Then apply brightness
                                    img_avg = img_avg.astype(np.float32) * brightness_factor
                                    img_avg = np.clip(img_avg, 0, 255).astype(np.uint8)
                                    img = Image.fromarray(img_avg)
                                except Exception as e:
                                    print(f"Error processing bright interpolated frame: {e}")
                                    img = self._get_fallback_frame(video_path)
                            else:
                                img = self._get_fallback_frame(video_path)
                        else:
                            img = self._get_fallback_frame(video_path)
                    
                    elif original_frame.startswith('bright_'):
                        # This is a brightness variant of another brightness variant
                        inner_parts = original_frame.split('_', 2)
                        if len(inner_parts) >= 3:
                            inner_brightness = float(inner_parts[1])
                            innermost_frame = inner_parts[2]
                            # Combine brightness factors
                            combined_brightness = brightness_factor * inner_brightness
                            
                            frame_path = os.path.join(video_path, innermost_frame)
                            if os.path.exists(frame_path):
                                try:
                                    img = Image.open(frame_path)
                                    img_np = np.asarray(img).astype(np.float32)
                                    img_np = img_np * combined_brightness
                                    img_np = np.clip(img_np, 0, 255).astype(np.uint8)
                                    img = Image.fromarray(img_np)
                                except Exception as e:
                                    print(f"Error applying combined brightness: {e}")
                                    img = self._get_fallback_frame(video_path)
                            else:
                                img = self._get_fallback_frame(video_path)
                        else:
                            img = self._get_fallback_frame(video_path)
                    
                    else:
                        # This is a brightness variant of a regular frame
                        frame_path = os.path.join(video_path, original_frame)
                        if os.path.exists(frame_path):
                            try:
                                img = Image.open(frame_path)
                                img_np = np.asarray(img).astype(np.float32)
                                img_np = img_np * brightness_factor
                                img_np = np.clip(img_np, 0, 255).astype(np.uint8)
                                img = Image.fromarray(img_np)
                            except Exception as e:
                                print(f"Error applying brightness to {frame_path}: {e}")
                                img = Image.open(frame_path)  # Fallback to original
                        else:
                            img = self._get_fallback_frame(video_path)
                else:
                    img = self._get_fallback_frame(video_path)
            
            else:
                # Regular frame
                frame_path = os.path.join(video_path, frame_name)
                if os.path.exists(frame_path):
                    try:
                        img = Image.open(frame_path)
                    except Exception as e:
                        print(f"Error loading frame {frame_path}: {e}")
                        img = Image.fromarray(np.zeros((224, 224, 3), dtype=np.uint8))
                else:
                    print(f"Frame not found: {frame_path}")
                    img = Image.fromarray(np.zeros((224, 224, 3), dtype=np.uint8))
            
            frames.append(img)
        
        # Ensure exactly clip_size frames
        while len(frames) < self.clip_size:
            if frames:
                frames.append(frames[-1])  # Repeat last frame
            else:
                frames.append(Image.fromarray(np.zeros((224, 224, 3), dtype=np.uint8)))
        
        frames = frames[:self.clip_size]
        return frames

    def _load_data(self):
        """Create single clip per video with new augmentation strategy"""
        data = []
        subfolders = sorted(os.listdir(self.root_dir))
        
        if len(subfolders) != 25:
            raise ValueError(f"Dataset must contain exactly 25 subfolders. Found {len(subfolders)}.")

        for label, action in enumerate(subfolders):
            action_path = os.path.join(self.root_dir, action)
            if not os.path.isdir(action_path):
                continue
                
            for video_folder in os.listdir(action_path):
                video_path = os.path.join(action_path, video_folder)
                if not os.path.isdir(video_path):
                    continue
                
                # Skip already processed folders
                if '_' in video_folder and video_folder.split('_')[-1].isdigit():
                    continue
                
                # Get all image frames (unsampled)
                all_frames = sorted([f for f in os.listdir(video_path) 
                                   if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))])
                
                if len(all_frames) == 0:
                    continue
                
                # Check if we need augmentation
                required_frames = self.frame_rate * self.clip_size  # e.g., 32 * 8 = 256
                
                if len(all_frames) < required_frames:
                    # Apply augmentation to get enough frames
                    augmented_frames = self._augment_frames_sequence(all_frames, required_frames)
                    working_frames = augmented_frames
                else:
                    working_frames = all_frames
                
                # Now sample from the working frames
                sampled_frames = working_frames[::self.frame_rate]
                
                # Create single clip per video
                if len(sampled_frames) >= self.clip_size:
                    # Take the first clip_size frames for the single clip
                    clip_frames = sampled_frames[:self.clip_size]
                else:
                    # This should be rare since we augmented first, but handle it
                    # Repeat frames to reach clip_size
                    while len(sampled_frames) < self.clip_size:
                        sampled_frames.extend(sampled_frames)
                    clip_frames = sampled_frames[:self.clip_size]
                
                # Add single clip to dataset
                data.append((video_path, label, 0, clip_frames))
                
        print(f"Total clips created: {len(data)} ")
        return data

    def __len__(self):
        return len(self.data)

    def _resolve_frame_path(self, video_path, frame_name):
        """Helper method to resolve frame path, handling brightness variants"""
        if frame_name.startswith('bright_'):
            parts = frame_name.split('_', 2)
            if len(parts) >= 3:
                original_frame = parts[2]
                frame_path = os.path.join(video_path, original_frame)
                if os.path.exists(frame_path):
                    return frame_path
        else:
            frame_path = os.path.join(video_path, frame_name)
            if os.path.exists(frame_path):
                return frame_path
        return None

    def _get_fallback_frame(self, video_path):
        """Get a fallback frame when specific frame loading fails"""
        available_frames = [f for f in os.listdir(video_path) 
                          if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
        if available_frames:
            return Image.open(os.path.join(video_path, available_frames[0]))
        else:
            return Image.fromarray(np.zeros((224, 224, 3), dtype=np.uint8))

    def _load_frames_from_clip(self, video_path, clip_frames):
        """Load frames for a specific clip"""
        frames = []
        
        for frame_name in clip_frames:
            if frame_name == 'black_frame':
                # Create black frame
                img = Image.fromarray(np.zeros((224, 224, 3), dtype=np.uint8))
            
            elif frame_name.startswith('interp_'):
                # Handle interpolated frames
                # Format: "interp_frame1_frame2"
                parts = frame_name.replace('interp_', '').split('_')
                if len(parts) >= 2:
                    frame1_name = '_'.join(parts[:-1])
                    frame2_name = parts[-1]
                    
                    frame1_path = os.path.join(video_path, frame1_name)
                    frame2_path = os.path.join(video_path, frame2_name)
                    
                    if os.path.exists(frame1_path) and os.path.exists(frame2_path):
                        try:
                            img1 = np.array(Image.open(frame1_path))
                            img2 = np.array(Image.open(frame2_path))
                            # Simple interpolation
                            img_avg = np.mean([img1, img2], axis=0, dtype=np.uint8)
                            img = Image.fromarray(img_avg)
                        except Exception as e:
                            print(f"Error interpolating frames: {e}")
                            img = Image.open(frame1_path)  # Fallback to first frame
                    else:
                        img = self._get_fallback_frame(video_path)
                else:
                    img = self._get_fallback_frame(video_path)
            
            elif frame_name.startswith('bright_'):
                # Handle brightness-varied frames - extract original frame and apply brightness
                # Format: "bright_1.2_original_frame.jpg"
                parts = frame_name.split('_', 2)  # Split into ['bright', '1.2', 'original_frame.jpg']
                if len(parts) >= 3:
                    brightness_factor = float(parts[1])
                    original_frame = parts[2]
                    
                    # Handle nested cases (bright of interp, bright of bright, etc.)
                    if original_frame.startswith('interp_'):
                        # This is a brightness variant of an interpolated frame
                        interp_parts = original_frame.replace('interp_', '').split('_')
                        if len(interp_parts) >= 2:
                            frame1_name = '_'.join(interp_parts[:-1])
                            frame2_name = interp_parts[-1]
                            
                            frame1_path = os.path.join(video_path, frame1_name)
                            frame2_path = os.path.join(video_path, frame2_name)
                            
                            if os.path.exists(frame1_path) and os.path.exists(frame2_path):
                                try:
                                    img1 = np.array(Image.open(frame1_path))
                                    img2 = np.array(Image.open(frame2_path))
                                    # Interpolate first
                                    img_avg = np.mean([img1, img2], axis=0, dtype=np.uint8)
                                    # Then apply brightness
                                    img_avg = img_avg.astype(np.float32) * brightness_factor
                                    img_avg = np.clip(img_avg, 0, 255).astype(np.uint8)
                                    img = Image.fromarray(img_avg)
                                except Exception as e:
                                    print(f"Error processing bright interpolated frame: {e}")
                                    img = self._get_fallback_frame(video_path)
                            else:
                                img = self._get_fallback_frame(video_path)
                        else:
                            img = self._get_fallback_frame(video_path)
                    
                    elif original_frame.startswith('bright_'):
                        # This is a brightness variant of another brightness variant
                        inner_parts = original_frame.split('_', 2)
                        if len(inner_parts) >= 3:
                            inner_brightness = float(inner_parts[1])
                            innermost_frame = inner_parts[2]
                            # Combine brightness factors
                            combined_brightness = brightness_factor * inner_brightness
                            
                            frame_path = os.path.join(video_path, innermost_frame)
                            if os.path.exists(frame_path):
                                try:
                                    img = Image.open(frame_path)
                                    img_np = np.asarray(img).astype(np.float32)
                                    img_np = img_np * combined_brightness
                                    img_np = np.clip(img_np, 0, 255).astype(np.uint8)
                                    img = Image.fromarray(img_np)
                                except Exception as e:
                                    print(f"Error applying combined brightness: {e}")
                                    img = self._get_fallback_frame(video_path)
                            else:
                                img = self._get_fallback_frame(video_path)
                        else:
                            img = self._get_fallback_frame(video_path)
                    
                    else:
                        # This is a brightness variant of a regular frame
                        frame_path = os.path.join(video_path, original_frame)
                        if os.path.exists(frame_path):
                            try:
                                img = Image.open(frame_path)
                                img_np = np.asarray(img).astype(np.float32)
                                img_np = img_np * brightness_factor
                                img_np = np.clip(img_np, 0, 255).astype(np.uint8)
                                img = Image.fromarray(img_np)
                            except Exception as e:
                                print(f"Error applying brightness to {frame_path}: {e}")
                                img = Image.open(frame_path)  # Fallback to original
                        else:
                            img = self._get_fallback_frame(video_path)
                else:
                    img = self._get_fallback_frame(video_path)
            
            else:
                # Regular frame
                frame_path = os.path.join(video_path, frame_name)
                if os.path.exists(frame_path):
                    try:
                        img = Image.open(frame_path)
                    except Exception as e:
                        print(f"Error loading frame {frame_path}: {e}")
                        img = Image.fromarray(np.zeros((224, 224, 3), dtype=np.uint8))
                else:
                    print(f"Frame not found: {frame_path}")
                    img = Image.fromarray(np.zeros((224, 224, 3), dtype=np.uint8))
            
            frames.append(img)
        
        # Ensure exactly clip_size frames
        while len(frames) < self.clip_size:
            if frames:
                frames.append(frames[-1])  # Repeat last frame
            else:
                frames.append(Image.fromarray(np.zeros((224, 224, 3), dtype=np.uint8)))
        
        frames = frames[:self.clip_size]
        return frames

    def __getitem__(self, idx):
        video_path, label, clip_idx, clip_frames = self.data[idx]
        
        # Load frames for this specific clip
        frames = self._load_frames_from_clip(video_path, clip_frames)
        
        # Apply transforms
        if self.transform:
            frames = [self.transform(frame) for frame in frames]
        
        return torch.stack(frames), label

def get_dataloader(root_dir, batch_size=8, clip_size=8, train_ratio=0.8, val_ratio=0.1, frame_rate=32):
    """
    Create dataloaders using single clip per video dataset with augmentation
    """
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Create virtual dataset
    dataset = MultiClipHMDBDataset(
        root_dir=root_dir, 
        clip_size=clip_size, 
        frame_rate=frame_rate,
        transform=transform
    )
    
    if len(dataset) == 0:
        raise ValueError("No valid clips found in dataset. Check your data structure and frame_rate parameter.")
    
    # Create splits
    indices = range(len(dataset))
    labels = [label for _, label, _, _ in dataset.data]
    
    # Check if we have enough samples for each class
    unique_labels, counts = np.unique(labels, return_counts=True)
    min_count = np.min(counts)
    print(f"Minimum samples per class: {min_count}")
    
    if min_count < 2:
        print("Some classes have only 1 sample. Using random split instead of stratified split.")
        # First split: train+val vs test
        train_val_idx, test_idx = train_test_split(
            indices, test_size=1-(train_ratio+val_ratio), random_state=42
        )
        
        # Second split: train vs val
        train_idx, val_idx = train_test_split(
            train_val_idx, test_size=val_ratio/(train_ratio+val_ratio), random_state=42
        )
    else:
        # First split: train+val vs test
        train_val_idx, test_idx = train_test_split(
            indices, test_size=1-(train_ratio+val_ratio), stratify=labels, random_state=42
        )
        
        # Second split: train vs val
        train_val_labels = [labels[i] for i in train_val_idx]
        train_idx, val_idx = train_test_split(
            train_val_idx, test_size=val_ratio/(train_ratio+val_ratio), 
            stratify=train_val_labels, random_state=42
        )
    
    # Create subsets
    train_dataset = torch.utils.data.Subset(dataset, train_idx)
    val_dataset = torch.utils.data.Subset(dataset, val_idx)
    test_dataset = torch.utils.data.Subset(dataset, test_idx)
    
    # Create dataloaders
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True, 
        num_workers=0, pin_memory=False
    )
    val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=batch_size, shuffle=False,
        num_workers=0, pin_memory=False
    )
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False,
        num_workers=0, pin_memory=False
    )
    
    print(f"Dataset splits: Train={len(train_dataset)}, Val={len(val_dataset)}, Test={len(test_dataset)}")
    return train_loader, val_loader, test_loader



In [40]:
def load_timesformer_model():
    """
    Load the pre-trained TimeSformer model for video classification.
    
    """
    import logging
    warnings.filterwarnings("ignore", message="Some weights of TimesformerForVideoClassification were not initialized")
    logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR) 
    
    label_index_dict={'brush_hair': 0, 'cartwheel': 1, 'catch': 2, 'chew': 3, 'climb': 4, 'climb_stairs': 5, 'draw_sword': 6, 'eat': 7, 'fencing': 8, 'flic_flac': 9, 'golf': 10, 'handstand': 11, 'kiss': 12, 'pick': 13, 'pour': 14, 'pullup': 15, 'pushup': 16, 'ride_bike': 17, 'shoot_bow': 18, 'shoot_gun': 19, 'situp': 20, 'smile': 21, 'smoke': 22, 'throw': 23, 'wave': 24}
    index_label_dict={0: 'brush_hair', 1: 'cartwheel', 2: 'catch', 3: 'chew', 4: 'climb', 5: 'climb_stairs', 6: 'draw_sword', 7: 'eat', 8: 'fencing', 9: 'flic_flac',10: 'golf', 11: 'handstand', 12: 'kiss', 13: 'pick', 14: 'pour', 15: 'pullup', 16: 'pushup', 17: 'ride_bike', 18: 'shoot_bow', 19: 'shoot_gun', 20: 'situp', 21: 'smile', 22: 'smoke', 23: 'throw', 24: 'wave'}
    # Load the processor and model from Hugging Face
    processor =AutoImageProcessor.from_pretrained("facebook/timesformer-base-finetuned-k400")
    ckpt = "facebook/timesformer-base-finetuned-k400"
    model = TimesformerForVideoClassification.from_pretrained(ckpt,label2id = label_index_dict,id2label = index_label_dict,ignore_mismatched_sizes = True)
    #model.classifier = torch.nn.Linear(model.config.hidden_size, 25)
    # Optionally load fine-tuned weights if available
    checkpoint_path = "/user/HS402/zs00774/Downloads/action-recognition-vit/timesformer_model.pth"  # Update this path if you have fine-tuned weights
    if checkpoint_path and os.path.exists(checkpoint_path):
        model.load_state_dict(torch.load(checkpoint_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
        print("Loaded fine-tuned weights from:", checkpoint_path)
    else:
        print("Using pre-trained TimeSformer weights.")

    return processor, model

In [41]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    
    if isinstance(predictions, torch.Tensor):
        predictions = predictions.cpu().numpy()
    if isinstance(labels, torch.Tensor):
        labels = labels.cpu().numpy()
    
    # Top-1 accuracy
    top1_predictions = np.argmax(predictions, axis=1)
    top1_accuracy = np.mean(top1_predictions == labels)
    
    # Top-5 accuracy
    top5_predictions = np.argsort(predictions, axis=1)[:, -5:]
    top5_correct = np.array([labels[i] in top5_predictions[i] for i in range(len(labels))])
    top5_accuracy = np.mean(top5_correct)
    
    return {
        "top1_accuracy": top1_accuracy,
        "top5_accuracy": top5_accuracy,
        "eval_top1_accuracy": top1_accuracy,
        "eval_top5_accuracy": top5_accuracy,
    }


In [42]:
def train_model_with_trainer(data_dir, epochs, batch_size, learning_rate):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    train_loader, val_loader, test_loader = get_dataloader(data_dir, batch_size)
    processor, model = load_timesformer_model()
    
    train_dataset = train_loader.dataset
    val_dataset = val_loader.dataset
    
    def data_collator(batch):
        videos = torch.stack([item[0] for item in batch])
        labels = torch.tensor([item[1] for item in batch])
        return {
            'pixel_values': videos,
            'labels': labels
        }
    optimizer = optim.SGD(model.parameters(), momentum= 0.9, weight_decay= 1e-3,
    			  lr= learning_rate)
    
    training_args = TrainingArguments(output_dir = "./results",
                                      overwrite_output_dir = True,
                                      eval_strategy = 'epoch',
                                      per_device_train_batch_size = 8,
                                      per_device_eval_batch_size = 8,
                                      num_train_epochs = epochs,
                                      logging_dir = os.path.normpath(os.path.join('./results', 'logs')),
                                      logging_strategy = "epoch",
                                      save_strategy = 'epoch',
                                      save_total_limit = 1,
                                      remove_unused_columns = False,
                                      load_best_model_at_end = True,
                                      metric_for_best_model = 'eval_top1_accuracy',
                                      greater_is_better = True,
                                      label_smoothing_factor = 0.1,
                                      report_to = "tensorboard",
                                      push_to_hub = False,
                                     save_only_model = True)
    
    early_stopping_callback = EarlyStoppingCallback(early_stopping_patience= 6 )
    
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        optimizers=(optimizer,None),
        eval_dataset=val_dataset,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        callbacks = [early_stopping_callback])
    
    print("Starting training...")
    trainer.train()
    
    print("\nEvaluating on test set...")
    test_dataset = test_loader.dataset
    test_results = trainer.evaluate(eval_dataset=test_dataset)
    
    print(f"\n=== FINAL TEST RESULTS ===")
    print(f"Test Top-1 Accuracy: {test_results.get('eval_top1_accuracy', 'N/A'):.4f}")
    print(f"Test Top-5 Accuracy: {test_results.get('eval_top5_accuracy', 'N/A'):.4f}")
    print(f"Test Loss: {test_results.get('eval_loss', 'N/A'):.4f}")
    
    trainer.save_model("./timesformer_model")
    
    with open("./test_results.json", "w") as f:
        json.dump(test_results, f, indent=2)
    
    print("Training complete. Model and results saved.")
    return test_results


In [43]:
def evaluate_saved_model(model_path, data_dir, batch_size=8):
    from transformers import TimesformerForVideoClassification
    
    model = TimesformerForVideoClassification.from_pretrained(model_path)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    _, _, test_loader = get_dataloader(data_dir, batch_size)
    
    def data_collator(batch):
        videos = torch.stack([item[0] for item in batch])
        labels = torch.tensor([item[1] for item in batch])
        return {
            'pixel_values': videos,
            'labels': labels
        }
    
    trainer = Trainer(
        model=model,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )
    
    results = trainer.evaluate(eval_dataset=test_loader.dataset)
    
    print(f"=== Model Evaluation: {model_path} ===")
    print(f"Top-1 Accuracy: {results.get('eval_top1_accuracy', 'N/A'):.4f}")
    print(f"Top-5 Accuracy: {results.get('eval_top5_accuracy', 'N/A'):.4f}")
    print(f"Loss: {results.get('eval_loss', 'N/A'):.4f}")
    
    return results


In [44]:
train_loader, val_loader, test_loader = get_dataloader(
    root_dir="/user/HS402/zs00774/Downloads/HMDB_simp",
    batch_size=8,
    clip_size=8,
    frame_rate=32
)

Total clips created: 1250 
Minimum samples per class: 50
Dataset splits: Train=999, Val=126, Test=125


In [46]:
train_model_with_trainer(
        data_dir="/user/HS402/zs00774/Downloads/HMDB_simp",
        epochs=20,
        batch_size=8,
        learning_rate=0.005
    )

Using device: cuda
Total clips created: 1250 
Minimum samples per class: 50
Dataset splits: Train=999, Val=126, Test=125
Using pre-trained TimeSformer weights.
Starting training...


Epoch,Training Loss,Validation Loss,Top1 Accuracy,Top5 Accuracy
1,2.039300,1.319268,0.793651,0.928571
2,0.957800,1.108467,0.841270,0.968254
3,0.773300,1.018350,0.888889,0.960317
4,0.693200,1.004570,0.880952,0.960317
5,0.659400,1.002734,0.888889,0.960317
6,0.643900,1.020644,0.880952,0.960317
7,0.637800,1.017763,0.888889,0.960317
8,0.634700,1.014736,0.888889,0.952381
9,0.633000,1.016719,0.888889,0.952381



Evaluating on test set...



=== FINAL TEST RESULTS ===
Test Top-1 Accuracy: 0.8720
Test Top-5 Accuracy: 1.0000
Test Loss: 0.9591
Training complete. Model and results saved.


{'eval_top1_accuracy': 0.872,
 'eval_top5_accuracy': 1.0,
 'eval_loss': 0.9590616822242737,
 'eval_runtime': 26.2949,
 'eval_samples_per_second': 4.754,
 'eval_steps_per_second': 0.608,
 'epoch': 9.0}